In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint


In [3]:
'''

This creates the extedned.csv file which contains the augmented images and their labels.

'''


df = pd.read_csv('train_images.csv')

# Define a function to generate augmented image paths
def generate_augmented_paths(row):
    original_path = row['image_path']
    augmented_paths = [
        original_path,
        original_path.replace('.jpg', '_flipped.jpg'),
        original_path.replace('.jpg', '_zoomed.jpg'),
        original_path.replace('.jpg', '_flipped_zoomed.jpg'),
        original_path.replace('.jpg', '_expanded.jpg'),
        original_path.replace('.jpg', '_expanded_and_zoomed.jpg'),
        original_path.replace('.jpg', '_expanded_and_flipped.jpg'),
        original_path.replace('.jpg', '_expanded_and_flipped_and_zoomed.jpg')
    ]
    return augmented_paths

# Apply the function to each row and create a list of augmented image paths
augmented_images = df.apply(generate_augmented_paths, axis=1)
augmented_paths = [path for paths in augmented_images for path in paths]

# Create a list of labels to match the length of augmented paths
labels = [label for label in df['label'] for _ in range(8)]  # Repeat each label 8 times for the 8 augmented paths

# Create a DataFrame with extended image paths and labels
extended_df = pd.DataFrame({'image_path': augmented_paths, 'label': labels})

# Save the extended dataframe to a new CSV
extended_df.to_csv('extended.csv', index=False)




In [ ]:
gpt_attributes = pd.read_csv('gpt_attributes.csv')
train_images = pd.read_csv('extended.csv')

In [5]:
train = pd.merge(gpt_attributes, train_images, on='label')
train.head()

,label,species,has_bill_shape,has_wing_color,has_upperparts_color,has_underparts_color,has_breast_pattern,has_back_color,has_tail_shape,has_upper_tail_color,...,has_shape,has_back_pattern,has_tail_pattern,has_belly_pattern,has_primary_color,has_leg_color,has_bill_color,has_crown_color,has_wing_pattern,image_path
0,1,Black_footed_Albatross,curved_down,black,black,white,solid,black,rounded_tail,black,...,upright-perching_water-like,solid,solid,solid,black,pink,black,black,solid,/train_images/1.jpg
1,1,Black_footed_Albatross,curved_down,black,black,white,solid,black,rounded_tail,black,...,upright-perching_water-like,solid,solid,solid,black,pink,black,black,solid,/train_images/1_flipped.jpg
2,1,Black_footed_Albatross,curved_down,black,black,white,solid,black,rounded_tail,black,...,upright-perching_water-like,solid,solid,solid,black,pink,black,black,solid,/train_images/1_zoomed.jpg
3,1,Black_footed_Albatross,curved_down,black,black,white,solid,black,rounded_tail,black,...,upright-perching_water-like,solid,solid,solid,black,pink,black,black,solid,/train_images/1_flipped_zoomed.jpg
4,1,Black_footed_Albatross,curved_down,black,black,white,solid,black,rounded_tail,black,...,upright-perching_water-like,solid,solid,solid,black,pink,black,black,solid,/train_images/1_expanded.jpg


In [6]:
#shuffle rows
train = train.sample(frac=1).reset_index(drop=True)

In [16]:
image_path = train['image_path']
image_path.head()


#replace with your path
path = '/Users/danielvlantis/Desktop/ISDS/Applied Machine Learning/aml-birds/train_images/'

image_path = path + image_path.astype(str)
image_path.head()

0    /Users/danielvlantis/Desktop/aml-birds/train_i...
1    /Users/danielvlantis/Desktop/aml-birds/train_i...
2    /Users/danielvlantis/Desktop/aml-birds/train_i...
3    /Users/danielvlantis/Desktop/aml-birds/train_i...
4    /Users/danielvlantis/Desktop/aml-birds/train_i...
Name: image_path, dtype: object

In [10]:
'''
Set the label to be the species or the attributes

'''
#initialise
label = train


#all atributes
label = train.drop(['image_path', 'label', 'species'], axis=1) #all attributes

#physical attributes
#label = label.drop(['has_bill_color','has_crown_color','has_underparts_color','has_upperparts_color','has_wing_color','has_back_color','has_breast_color','has_eye_color','has_forehead_color','has_nape_color','has_primary_color','has_leg_color','has_belly_color','has_under_tail_color','has_upper_tail_color','has_eye_color','has_throat_color'], axis=1)

#color attributes
#label = label[['has_bill_color','has_crown_color','has_underparts_color','has_upperparts_color','has_wing_color','has_back_color','has_breast_color','has_eye_color','has_forehead_color','has_nape_color','has_primary_color','has_leg_color','has_belly_color','has_under_tail_color','has_upper_tail_color','has_eye_color','has_throat_color']]

#for predicting only the label
#label = train['label']

In [13]:
label_dummies = pd.get_dummies(label).astype(int)
columns = label_dummies.shape[1] #number of columns

In [15]:
'''
This function converts each image to the same dimensions: 256x256x3

'''

#initialize empty array
image_array = np.zeros((len(image_path), 256, 256, 3), dtype='uint8')

#load images into array
for i in tqdm(range(len(image_path))):
    image = Image.open(image_path[i])
    if image.mode != 'RGB':
        image = image.convert('RGB')

    image = Image.open(image_path[i])
    image = image.resize((256,256))

    if np.array(image).shape != (256, 256, 3):
        image = image.convert('RGB')

    image_array[i] = np.array(image)

image_array.shape



 25%|██▌       | 7924/31408 [00:05<00:17, 1349.87it/s]


KeyboardInterrupt: 

In [18]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(image_array, label_dummies, test_size=0.2, random_state=42)


In [22]:

model = Sequential()
model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(columns, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

#set early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

#set model checkpoint
checkpoint = ModelCheckpoint(filepath='new_best_model.hdf5', save_best_only=True, save_weights_only=False)

#fit model
#without early stopping
#history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[checkpoint])

#use if early stopping is used
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping, checkpoint])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 127, 127, 32)      0         
                                                                 
 conv2d_4 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 62, 62, 64)       

KeyboardInterrupt: 

In [ ]:
#save model
model.save('bird_cnn.h5')

In [23]:
#plot accuracy and loss
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [24]:
#load test images
test_images = pd.read_csv('test_images_path.csv')
test_images.head()

,id,image_path,label
0,1,/test_images/999.jpg,1
1,2,/test_images/998.jpg,1
2,3,/test_images/997.jpg,1
3,4,/test_images/996.jpg,1
4,5,/test_images/995.jpg,1


In [ ]:
'''
Convert test images to the same dimensions as the training images: 256x256x3
'''

test_images = test_images['image_path']

#replace with your test path
test_path = '/Users/danielvlantis/Desktop/ISDS/Applied Machine Learning/aml-birds/test_images/'


test_images =  test_path+ test_images.astype(str)
test_images.head()

#initialize empty array
test_image_array = np.zeros((len(test_images), 256, 256, 3), dtype='uint8')

#load images into array
for i in tqdm(range(len(test_images))):
    if image.mode != 'RGB':
        image = image.convert('RGB')

    image = Image.open(test_images[i])
    image = image.resize((256,256))

    if np.array(image).shape != (256, 256, 3):
        image = image.convert('RGB')

    test_image_array[i] = np.array(image)

test_image_array.shape

In [ ]:
test_predictions = model.predict(test_image_array)

In [ ]:
#For label approach (not attributes)
test_predictions = np.argmax(test_predictions, axis=1)

#convert test_predictions to pd series
test_predictions_pd = pd.Series(test_predictions)
test_predictions_pd.head()

#create pd series with numbers from 1 to length of test_predictions
id = pd.Series(range(1, len(test_predictions)+1))
id.head()
#combine id and test_predictions_pd into dataframe
test_predictions_pd = pd.concat([id, test_predictions_pd], axis=1)
test_predictions_pd.head()

#rename columns to id and label
test_predictions_pd.columns = ['id', 'label']
test_predictions_pd.head()

#save to csv
test_predictions_pd.to_csv('submission.csv', index=False, header=['id', 'label'])